In [ ]:
%pip install -e . --quiet

This blog post describes three use cases of the `fumbbl_replays` package.
For the first use case, the functionality of the package is sufficient, with no additional programming needed.
For the other two use cases, additional programming is needed.
In the future, this code could be turned into a function and added to the package.

# Application 1: What's the play



In [ ]:
import sys
from PIL import Image, ImageDraw

with Image.open("doc/Whats the play 04 - Dwarf Vs Elven Union.png") as im:
   display(im)

Lets try to reproduce this plot with `fumbbl_replays`.
We start with fetching a Dwarf Roster. 
Then create the positions including playerStates (i.e. the Stunned Troll Slayer). 
Then finally we add the skills.

In [ ]:
import fumbbl_replays as fb

my_roster = fb.fetch_roster("Dwarf")

my_setup = ['setup', ['B3: d14', 'TS4: b15X', 'L10: l17', 
                      'L11: n17', 'L9: j19', 'L5: l20', 'L8: n20', 
                      'L6: j21', 'R1: m21o', 'L7: l22', 'B2: n22']]

positions = fb.create_position(my_roster, my_setup)

fb.add_skill_to_player(positions, "R1", "On the Ball")
fb.add_skill_to_player(positions, "B3", "Guard")
#fb.add_skill_to_player(positions, "TS4", "Block") # TS already comes with block
fb.add_skill_to_player(positions, "TS4", "Mighty Blow")
fb.add_skill_to_player(positions, "L7", "Guard")
fb.add_skill_to_player(positions, "L8", "Guard")
fb.add_skill_to_player(positions, "L9", "Guard")
fb.add_skill_to_player(positions, "L10", "Guard")

Repeat for Elven Union:

In [ ]:
import pandas as pd

my_roster = fb.fetch_roster("Elven Union")

my_setup = ['setup', ['T1: l25', 'C3: i24', 'B6: i23', 
                      'B7: g22', 'L10: i21', 'L11: h20', 'L12: c16', 
                      'L8: c15', 'C2: h11']]

positions2 = fb.create_position(my_roster, my_setup, home_away = 'teamAway')

fb.add_skill_to_player(positions2, "T1", "Accurate")
fb.add_skill_to_player(positions2, "T1", "Leader")
fb.add_skill_to_player(positions2, "C3", "Dodge")
fb.add_skill_to_player(positions2, "C3", "Block")
fb.add_skill_to_player(positions2, "B6", "Dodge")
fb.add_skill_to_player(positions2, "B6", "Strip Ball")
fb.add_skill_to_player(positions2, "B7", "Mighty Blow")
fb.add_skill_to_player(positions2, "L10", "Block")

positions = pd.concat([positions, positions2])

In [ ]:
fb.create_plot(positions, red_team = "teamAway", orientation = 'H', skill_bands = True)

We can reposition the Elven Union players to describe a possible solution to the what's the play puzzle.
This facilitates discussions of alternative plays, and the risks and benefits associated with them.

In [ ]:
positions = fb.move_piece(positions, "teamAway", "T1", "n25")
positions = fb.move_piece(positions, "teamAway", "B6", "n24")
positions = fb.move_piece(positions, "teamAway", "C3", "k25")
positions = fb.move_piece(positions, "teamAway", "B7", "k24")
positions = fb.move_piece(positions, "teamAway", "L10", "h24")
positions = fb.move_piece(positions, "teamAway", "L11", "h25")
positions = fb.move_piece(positions, "teamAway", "L12", "g22")
positions = fb.move_piece(positions, "teamAway", "C2", "g19")

fb.create_plot(positions, red_team = "teamAway", orientation = 'H', skill_bands = True)

The remaining lineman L8 could finally do a dodge and two rushes to end up in g23.

This also immediately gives rise to new feature requests: highlighting squares, drawing arrows, some form of pathfinding (e.g. find shortest path without dice rolling).

# Application 2: Plotting defensive tournament setups

Here the idea is to visualize defensive setups in a tournament setting.

To test the idea, I chose the Tilean Team Cup. It was an online NAF tournament in 2023 on FUMBBL that used the World Cup ruleset.
https://member.thenaf.net/index.php?module=NAF&type=tournaments&func=view&id=7495

NAF tournament director Stimme wrote:

https://www.thenaf.net/2023/05/tournament-director-blog-may-2023/

*Among the individual coaches, Siggi stood out with his Amazons, earning the title of best coach with a flawless record of six wins. Kurjo’s Orcs secured second place, while helborg’s Dark Elves came in third, both with five wins and a draw.* 


In [ ]:
import pandas as pd
import fumbbl_replays as fb

# point this to the location of the HDF5 datasets
path_to_datasets = '../fumbbl_datasets/datasets/current/'

# FUMBBL matches
target = 'df_matches.csv'
df_matches = pd.read_csv(path_to_datasets + target) 

# # subset on tilean team cup
df_matches = df_matches.query('tournament_id == 59383')

my_vec = df_matches['coach1_ranking'] == 'siggi'
my_vec2 = df_matches['coach2_ranking'] == 'siggi'

df_matches = df_matches[(my_vec + my_vec2)]

tilean_replays = df_matches['match_id'].values

In [ ]:
fullrun = 1

if fullrun:
    id = []
    match_ids = []
    race_defense = []
    race_offense = []

    for match_id in tilean_replays:
        match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id) 
        plot = fb.write_plot(match_id, positions, receiving_team, metadata, refresh = True, verbose = True)
        id.append(int(replay_id))
        match_ids.append(int(match_id))
        race_defense.append(metadata[4])
        race_offense.append(metadata[5])

    df_replays = pd.DataFrame( {"matchId": match_ids,
                                "replayId": id,
                                "raceOffense": race_offense,
                                "raceDefense": race_defense})
    target = 'kickoff_pngs/df_replays'
    df_replays.to_csv(target + '.csv', index = False)
else:
    # FUMBBL matches
    target = 'kickoff_pngs/df_replays.csv'
    df_replays = pd.read_csv(target)  

In [ ]:
metadata

In [ ]:
from PIL import Image
Image.open("kickoff_pngs/wood_elf/1606445_4447434_kickoff_lower_defense.png")

In [ ]:
my_replay = fb.fetch_replay(match_id = 4447483)
rosters = fb.extract_rosters_from_replay(my_replay)

(rosters
 .query('race == "Amazon"')
 .filter(['short_name', 'positionName', 'playerName', 'skillArrayRoster', 'learned_skills', 'cost', 'recoveringInjury'])
)

# Application 3: Roster development in league play



In [ ]:
import pandas as pd
import fumbbl_replays as fb

pd.set_option('display.max_colwidth', None)

# We do gnomes
matches = [4542630, 4543329, 4543359, 4544140, 4545345, 4545914, 4546710, 4546875, 4546908, 4548520, 4548537, 4548898, 4549961, 4549984, 4550284]
team_id = '1177218'

# top welf too
#matches = [4543274, 4543281, 4543501, 4543513, 4543524, 4551199, 4551662, 4551696, 4551850, 4551912, 4551947, 4551983, 4552023, 4552066, 4552172]
#team_id = '1176759'

i = 0

for match_id in matches:
    my_replay = fb.fetch_replay(match_id)
    df_positions = fb.extract_rosters_from_replay(my_replay) 
    df_positions = (df_positions
                    .query("teamId == @team_id")
                    .filter(['short_name', 'positionName', 'playerName', 'skillArrayRoster', 'learned_skills', 'cost', 'recoveringInjury'])
                    )
    df_positions['match_count'] = i+1
    if i == 0:
        res = df_positions
    else:
        res = pd.concat([res, df_positions])
    i = i + 1

    



In [ ]:
obj = res.query("match_count == 15 & recoveringInjury == 'None'").agg({'cost' : sum})
obj + 200000 + 110000 # cost plus 3x rerolls and apo plus skill value increases

In [ ]:
obj = res.query("match_count == 15 & recoveringInjury == 'None'")
obj

In [ ]:
player_names = res.query("match_count == 1")['playerName'].values

(res.query('playerName in @player_names')
 .pivot(index = ['playerName', 'positionName', 'cost'], columns = 'match_count', values = 'learned_skills')
 .sort_values(by=['cost'], ascending=False)
 .fillna(value = '-')
)